#Encoder


## Importación de librerías y datos

Por medio de nuestra libería ESIOS_contoller.py importamos nuestro último dataset de datos y lo parseamos para su uso. Sirve tanto como para Drive como jupiter.

In [76]:
import json, urllib, datetime, pickle, time
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.metrics import *
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
from keras.optimizers import *
from scipy.stats import *
from importlib.machinery import SourceFileLoader
from math import sqrt


try:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/TFM/01.Utils/ESIOS_contoller.py'
  in_colab = True
except:
  path = '../utils/ESIOS_contoller.py'
  in_colab = False
  

esios_assembler = SourceFileLoader('esios', path).load_module()

esios_controller = esios_assembler.ESIOS(in_colab)
data_consumo = esios_controller.get_data()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mostrando los datos de data_total.csv
(30555, 30)
________________________________________________________________________________


## Preparación de los datos

In [20]:
x_data_grouped = esios_controller.get_df_daily()
y_data_grouped = esios_controller.get_df_daily_all_day_prices()

x_data_grouped.head()

Mostrando los datos de data_total.csv
(30555, 30)
________________________________________________________________________________
Mostrando los datos de data_total.csv
(30555, 30)
________________________________________________________________________________


,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Precio mercado SPOT Diario_x,Demanda real,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,Precio SPOT PT,Precio SPOT FR,PVPC-target
fecha,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-09-30 00:00:00+00:00,117.430000,68.850000,68.725000,1.443652e+09,24267.750000,951.333333,6013.000000,288.500000,33.000000,254.916667,582.166667,24065.916667,23843.000000,47.480000,0.000000,60.385000,49.130000,55.075000,24632.000000,231.650000,-43.550000,-859.550000,-351.400000,1232.100000,1200.000000,55.075000,34.915000,122.640000
2015-10-01 00:00:00+00:00,124.598571,100.728571,101.031429,1.443699e+09,27736.230159,2297.317460,6006.492063,1694.936508,949.166667,745.301587,604.031746,27682.936508,27731.333333,47.391429,0.000000,56.811905,43.816190,57.638571,27700.190476,1643.461905,168.842857,-487.209524,-322.428571,525.380952,777.619048,57.638571,42.389048,121.936190
2015-10-02 00:00:00+00:00,122.746957,99.568261,99.987826,1.443784e+09,27762.130435,2745.507246,5946.210145,1715.507246,1014.927536,699.934783,612.521739,27747.376812,27713.304348,46.550000,0.065217,54.445217,43.578696,56.701739,27643.217391,1691.608696,729.826087,-512.604348,-260.304348,590.160870,1200.304348,56.701739,45.823043,114.612174
2015-10-03 00:00:00+00:00,114.161667,93.284583,93.995417,1.443872e+09,24795.368056,2520.423611,5632.680556,655.743056,589.041667,66.173611,604.756944,24796.375000,24776.958333,46.550000,0.770833,54.480417,44.911667,49.893333,24779.541667,866.587500,708.716667,-400.829167,-500.000000,471.208333,1200.000000,49.893333,38.094167,101.120833
2015-10-04 00:00:00+00:00,101.120833,80.730833,81.195833,1.443958e+09,23230.319444,9189.333333,4897.076389,506.173611,476.125000,29.548611,574.076389,23192.798611,23045.625000,46.791667,0.916667,46.652083,17.298750,36.721667,23175.583333,591.112500,1222.762500,-191.433333,-504.820833,1414.329167,655.462500,36.721667,36.074167,106.640000


In [0]:
columns_array = ['h'+str(i) for i in range(24)]

y_data_grouped = pd.DataFrame(y_data_grouped)
y_data_grouped = pd.DataFrame(y_data_grouped['PVPC_DEF'].values.tolist(), columns=columns_array)

In [22]:
y_data_grouped.tail()

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,h11,h12,h13,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
1396,109.97,111.44,109.74,108.56,105.29,103.77,106.16,109.52,109.63,110.14,110.37,109.36,104.78,101.28,99.91,99.60,99.82,100.47,103.66,107.56,107.17,111.19,105.61,NaN
1397,102.29,103.25,103.29,103.15,102.91,102.99,105.10,106.30,105.90,108.27,105.27,103.13,102.94,115.84,119.56,118.10,116.71,108.58,NaN,NaN,NaN,NaN,NaN,NaN
1398,101.31,100.54,100.90,110.91,116.41,114.26,113.77,115.77,110.74,106.74,106.00,105.10,106.00,106.66,105.58,106.97,109.70,104.77,105.53,100.37,NaN,NaN,NaN,NaN
1399,99.17,98.99,99.08,99.10,100.41,106.03,108.07,110.41,109.16,110.65,114.08,115.17,114.31,114.30,113.36,112.49,110.67,108.67,110.10,111.29,111.95,107.60,111.96,103.1
1400,99.79,99.25,99.02,104.07,108.60,110.51,109.84,110.68,110.93,111.92,110.12,109.93,109.86,108.27,109.20,110.68,105.84,112.39,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
y_data_grouped = y_data_grouped.bfill()
y_data_grouped = y_data_grouped.ffill()
y_data_grouped.tail()

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,h11,h12,h13,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
1396,109.97,111.44,109.74,108.56,105.29,103.77,106.16,109.52,109.63,110.14,110.37,109.36,104.78,101.28,99.91,99.60,99.82,100.47,103.66,107.56,107.17,111.19,105.61,103.1
1397,102.29,103.25,103.29,103.15,102.91,102.99,105.10,106.30,105.90,108.27,105.27,103.13,102.94,115.84,119.56,118.10,116.71,108.58,105.53,100.37,111.95,107.60,111.96,103.1
1398,101.31,100.54,100.90,110.91,116.41,114.26,113.77,115.77,110.74,106.74,106.00,105.10,106.00,106.66,105.58,106.97,109.70,104.77,105.53,100.37,111.95,107.60,111.96,103.1
1399,99.17,98.99,99.08,99.10,100.41,106.03,108.07,110.41,109.16,110.65,114.08,115.17,114.31,114.30,113.36,112.49,110.67,108.67,110.10,111.29,111.95,107.60,111.96,103.1
1400,99.79,99.25,99.02,104.07,108.60,110.51,109.84,110.68,110.93,111.92,110.12,109.93,109.86,108.27,109.20,110.68,105.84,112.39,110.10,111.29,111.95,107.60,111.96,103.1


In [57]:
# Split the data
x_data_grouped = pd.DataFrame(x_data_grouped[['PVPC_DEF', 'Holiday']])
x_train, x_valid, y_train, y_valid = train_test_split(x_data_grouped, y_data_grouped, test_size=0.1, shuffle=False)

print('Xtrain_dim:', x_train.shape)
print('Ytrain_dim:', y_train.shape)

Xtrain_dim: (1260, 2)
Ytrain_dim: (1260, 24)


## Modelo

Podemos observar que la capa de salida consta con una dimensión de 24, que equivale a las 24 horas del día. El modelo, que es la parte del encoder del autoencoder, esta pensado para que le entren dos valores de entrada (El precio medio diario y si es festivo) y con estos valores estimar el precio de las 24 horas del día. 

In [25]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.regularizers import l2


encoding_dim = 24

model = Sequential()
model.add(Dense(40, activation='relu',
                input_shape=(2,)))
model.add(Dense(output_dim=24,
                activation='linear'))
opt = SGD(lr=0.01, momentum=0.9, decay=1e-6)
model.compile(optimizer='adam', loss='mse', metrics=['mse','mae','mape'])
model.summary()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                120       
_________________________________________________________________
dense_2 (Dense)              (None, 24)                984       
Total params: 1,104
Trainable params: 1,104
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=24)`
  if sys.path[0] == '':


In [26]:
model.fit(x_train, y_train,epochs=300, 
                          shuffle=False,
                          verbose=1)



Epoch 1/300
1260/1260 [==============================] - 5s 4ms/step - loss: 11870.9695 - mean_squared_error: 11870.9695 - mean_absolute_error: 106.3285 - mean_absolute_percentage_error: 92.3632
Epoch 2/300
1260/1260 [==============================] - 0s 137us/step - loss: 8033.0493 - mean_squared_error: 8033.0493 - mean_absolute_error: 86.4324 - mean_absolute_percentage_error: 75.0929
Epoch 3/300
1260/1260 [==============================] - 0s 140us/step - loss: 4298.2088 - mean_squared_error: 4298.2088 - mean_absolute_error: 61.0399 - mean_absolute_percentage_error: 53.0335
Epoch 4/300
1260/1260 [==============================] - 0s 128us/step - loss: 1647.2368 - mean_squared_error: 1647.2368 - mean_absolute_error: 35.5313 - mean_absolute_percentage_error: 30.8351
Epoch 5/300
1260/1260 [==============================] - 0s 138us/step - loss: 465.9254 - mean_squared_error: 465.9254 - mean_absolute_error: 17.3953 - mean_absolute_percentage_error: 15.1066
Epoch 6/300
1260/1260 [======

In [27]:
x_valid[:1]

,PVPC_DEF,Holiday
fecha,,
2019-03-13 00:00:00+00:00,110.17,0.0


In [28]:
print(model.predict(x_valid[2:3]))

[[116.88188  114.639046 113.85359  113.968414 117.75668  121.80178
  125.10144  124.31095  123.99863  123.84987  122.604675 121.700714
  121.33302  120.60791  120.39877  121.772675 122.60213  123.61554
  124.6369   124.41562  123.90457  121.539276 121.080025 117.72576 ]]


## Métricas

In [65]:
y_valid = np.array(y_valid).flatten()
y_pred = np.array(model.predict(x_valid)).flatten()
esios_controller.get_metrics(y_valid, y_pred)

MSE:  25.348616239650763
RMSE:  5.034740930738221
MAE:  3.7718499370782372
Log_MSE:  0.0021309336462429665
MedianAE:  2.853354339599605
MAPE:  3.4275085536338796 %
SMAPE:  3.412800707999166 %
Variance:  0.5939009087689956
Max Error:  30.689855041503904


## Save to use

In [79]:
esios_controller.save_keras_model('/content/drive/My Drive/TFM/01.Utils/data', model,'Encoder', False, True)

Saved model to disk as /content/drive/My Drive/TFM/01.Utils/data/Encoder


(True, '/content/drive/My Drive/TFM/01.Utils/data')